In [1]:
import pandas as pd
import numpy as np
import lightgbm

label_address = "./data/label.csv"
test_address = "./data/test_set.csv"
train_address = "./data/train_set.csv"
orders_address = "./data/orders.csv"

train = pd.read_csv(train_address)
label = pd.read_csv(label_address)
test = pd.read_csv(test_address)
orders = pd.read_csv(orders_address)

test2 = orders[orders.eval_set =='test']
label["label"] = label["label"].astype(np.float32)
label = label["label"].values.tolist()

param = {"user_orders_num","user_history_item_num","user_history_different_item_num",
        "user_order_frequency_bydays","user_average_order","order_hour_of_day","days_since_prior_order",
         "day_weight","aisle_id","department_id","product_ordered_num","product_reordered_num",
        "product_reordered_rate","userproduct_order_num","userproduct_order_rate","userproduct_reorder_rate"}

train_model = lightgbm.Dataset(train[param],label = label,categorical_feature=['aisle_id', 'department_id'])
param2 = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    'num_leaves': 96,
    'max_depth': 10,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.95,
    'bagging_freq': 5
}
print(0)
model = lightgbm.train(param2,train_model,100)
print(1)
result = model.predict(test[param])
print(2)
TRESHOLD = 0.22 
test["result"] = result

d = dict()
for row in test.itertuples():
    if row.result > TRESHOLD:
        try:
            d[row.order_id] += ' ' + str(row.product_id)
        except:
            d[row.order_id] = str(row.product_id)

for order in test2.order_id:
    if order not in d:
        d[order] = 'None'

submission = pd.DataFrame.from_dict(d, orient='index')

submission.reset_index(inplace=True)
submission.columns = ['order_id', 'products']
submission.to_csv('submission.csv', index=False)

/Users/yuki/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


0


/Users/yuki/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


1
2


In [2]:
ligbm = pd.read_csv("./submission.csv", index_col=0)
ligbm

,products
order_id,
2774568,21903 21137 47766 42265 16797
1528013,21903
1376945,44632 14947 13176
1356845,13176
2161313,37059 196 10441 12427 37710 48142 1747 14715
1416320,21903 21137 24852 30391
1980631,13575 6184 9387 46061 13914 41400 22362
3202221,21137 24852 27966
3222866,40706 13187 35458 37507 37131 32912 49680 7969...
